In [94]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

In [95]:
import glob
glob.glob(r'C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\*.xlsx')

['C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_01-02-2024.xlsx',
 'C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_01-09-2024.xlsx',
 'C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_01-16-2024.xlsx',
 'C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_01-24-2024.xlsx',
 'C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_02-02-2024.xlsx',
 'C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_02-05-2024.xlsx',
 'C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_08-22-2023.xlsx',
 'C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_08-29-2023.xlsx',
 'C:\\Users\\aquirart\\Documents\\Pandas\\Case Assist Data\\test results-selected\\test_result_0

In [96]:
all_dfs_coveo = []
for one_filename in glob.glob(r'C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\*.xlsx'):
    print(f'Loading {one_filename}')
    new_df_coveo = pd.read_excel(one_filename,
                usecols=['Case Number', 'Coveo Final Classification Topic-Category-Subcategory'  ], 
                sheet_name='Result')
    all_dfs_coveo.append(new_df_coveo)

Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_01-02-2024.xlsx
Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_01-09-2024.xlsx
Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_01-16-2024.xlsx
Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_01-24-2024.xlsx
Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_02-02-2024.xlsx
Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_02-05-2024.xlsx
Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_08-22-2023.xlsx
Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_08-29-2023.xlsx
Loading C:\Users\aquirart\Documents\Pandas\Case Assist Data\test results-selected\test_result_09-05-2023.xlsx
Loading C:

In [97]:
df_coveo = pd.concat(all_dfs_coveo)

In [98]:
df_coveo[['Topic', 'Category','SubCategory']] = df_coveo['Coveo Final Classification Topic-Category-Subcategory'].str.split('|', expand=True)

In [99]:
all_dfs_case = []
for one_filename in glob.glob(r'C:\Users\aquirart\Documents\Pandas\Case Data\Case Data*.csv'):
    print(f'Loading {one_filename}')
    new_df_case = pd.read_csv(one_filename,
                usecols=['Id', 'CaseNumber', 'Original_Topic__c', 'Original_Category__c', 'Original_Subcategory__c', 
                'Subject', 'Description', 'SuppliedEmail','Origin', 'Type', 'Submitter_ID__c', 'CreatedDate', 
                'Topic__c', 'Category__c', 'Subcategory__c' ], 
                encoding='ISO-8859-1')
    all_dfs_case.append(new_df_case)

Loading C:\Users\aquirart\Documents\Pandas\Case Data\Case Data 2015_2016.csv


C:\Users\aquirart\AppData\Local\Temp\ipykernel_29968\1425215716.py:4: DtypeWarning: Columns (69,200,203,204,205) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df_case = pd.read_csv(one_filename,


Loading C:\Users\aquirart\Documents\Pandas\Case Data\Case Data 2017.csv


C:\Users\aquirart\AppData\Local\Temp\ipykernel_29968\1425215716.py:4: DtypeWarning: Columns (203,204,205) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df_case = pd.read_csv(one_filename,


Loading C:\Users\aquirart\Documents\Pandas\Case Data\Case Data 2018.csv
Loading C:\Users\aquirart\Documents\Pandas\Case Data\Case Data 2019.csv


C:\Users\aquirart\AppData\Local\Temp\ipykernel_29968\1425215716.py:4: DtypeWarning: Columns (204) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df_case = pd.read_csv(one_filename,


Loading C:\Users\aquirart\Documents\Pandas\Case Data\Case Data 2020.csv
Loading C:\Users\aquirart\Documents\Pandas\Case Data\Case Data 2021.csv
Loading C:\Users\aquirart\Documents\Pandas\Case Data\Case Data 2022.csv
Loading C:\Users\aquirart\Documents\Pandas\Case Data\Case Data 2023_2024.csv


In [100]:
df_case = pd.concat(all_dfs_case)

In [101]:
df_main = pd.merge(df_coveo[['Case Number', 'Coveo Final Classification Topic-Category-Subcategory', 'Topic', 'Category','SubCategory' ]],
                    df_case[['CaseNumber','Original_Topic__c', 'Original_Category__c', 'Original_Subcategory__c',
                    'Subject', 'Description', 'SuppliedEmail','Origin', 'Type', 'Submitter_ID__c', 'CreatedDate', 
                    'Topic__c', 'Category__c', 'Subcategory__c']], 
                    left_on = 'Case Number', 
                    right_on = 'CaseNumber', 
                    how='left' )
df_main = df_main.fillna('')

In [102]:
condidtions = [
    (df_main['Description'].str.contains('|'.join(['Death','BCC','died','Deceased','Passed Away']))) | (df_main['Subject'].str.contains('|'.join(['Death','BCC','died','Deceased','Passed Away']))),
    (df_main['Description'].str.contains('|'.join(['Settlement','Severance','arbitration','Separation Agreement']))) | (df_main['Subject'].str.contains('|'.join(['Settlement','Severance','arbitration','Separation Agreement']))) | (df_main['Original_Category__c'] == 'Settlement Agreement'),
    (df_main['SuppliedEmail'].str.contains('|'.join(['@LFG.com','@lfg.com']))) | (df_main['SuppliedEmail'].str.contains('|'.join(['@prudential.com','@securemail-lfg.com']))) | (df_main['Origin'] == 'Email'),
    (df_main['Original_Topic__c'] == 'Payroll') & (df_main['Original_Category__c'] == 'Expedited Pay Request'),
    (df_main['Type'] == 'Vendor') & (df_main['Original_Category__c'] == 'Supplemental AFLAC Plans'),
    (df_main['Type'] == 'COE') & (df_main['Original_Category__c'] == 'Supplemental AFLAC Plans'),
    (df_main['Description'].str.contains('|'.join(['Former Employee','FEP','Stale Request','Former EE']))) | (df_main['Subject'].str.contains('|'.join(['Former Employee','FEP','Stale Request','Former EE']))),
    (df_main['Description'].str.contains('|'.join(['Cal HR','CalHR','Loan Forgiveness','WVOE','Student Verification','Perkins','employment verification','VOE','work number','PSLF','verification request', 'Income verification','TWN' ]))) | (df_main['Subject'].str.contains('|'.join(['Cal HR','CalHR','Loan Forgiveness','WVOE','Student Verification','Perkins','employment verification','VOE','work number','PSLF','verification request', 'Income verification','TWN']))),
    (df_main['Original_Topic__c'] == 'Payroll') & (df_main['Original_Category__c'] == 'Quality Care Unit Payroll'),
    ((df_main['Original_Topic__c'] == 'Payroll') & (df_main['Original_Category__c'] == 'Overpayment Recovery')) | ((df_main['Description'].str.contains('|'.join(['Overpayment','repayment agreement','repayment plan']))) | (df_main['Subject'].str.contains('|'.join(['Overpayment','repayment agreement','repayment plan'])))),
    ((df_main['Original_Topic__c'] == 'UCPath Online') & (df_main['Original_Category__c'].isin(['UCPath Online Access','UCPath Online Inquiry']))) | ((df_main['Description'].str.contains('|'.join(['Portal','UCPath online']))) | (df_main['Subject'].str.contains('|'.join(['Portal','UCPath online'])))),
    (df_main['Description'].str.contains('|'.join(['Accellion','Data Breach','Breach','Cyber Attack','Accellion\'s File Transfer Appliance']))) | (df_main['Subject'].str.contains('|'.join(['Accellion','Data Breach','Breach','Cyber Attack','Accellion\'s File Transfer Appliance']))),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_PYBN']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Original_Topic__c'] == 'Payroll') & (df_main['Original_Category__c'] == 'Dashboard/Interface Inquiry'),
    (df_main['Original_Topic__c'] == 'Payroll') & (df_main['Original_Category__c'] == 'Tax Withholding Inquiry') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Original_Topic__c'] == 'Payroll') & (df_main['Original_Category__c'] == 'Tax Withholding Inquiry'),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_WFA']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_GL']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Benefits Election Inquiry') & ((df_main['Description'].str.contains('|'.join(['Change election for Health FSA and/or DepCare FSA']))) | (df_main['Subject'].str.contains('|'.join(['Request COVID-19 FSA Event'])))),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Quality Care Unit Benefits'),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Submit Form - Return From Leave Benefits Reinstatement'),
    ((df_main['Original_Topic__c'] == 'General Ledger') | (df_main['Original_Topic__c'] == 'Reporting')) & (df_main['Original_Category__c'] == 'Quality Care Unit Reporting'),
    (df_main['Original_Topic__c'] == 'Workforce Administration') & (df_main['Original_Category__c'] == 'Quality Care Unit Workforce Administration'),
    (df_main['Original_Category__c'].isin(['Health and Welfare Benefits Inquiry','Retirement/RASC Inquiry']) ),
    (df_main['Original_Topic__c'] == 'UCPath Help/Training Feedback') & (df_main['Original_Category__c'] == 'General Feedback'),
    (df_main['Original_Topic__c'] == 'UCPath Help/Training Feedback'),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Benefits Billing') & (df_main['Original_Subcategory__c'] == 'Benefits Billing Payment Plan'),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Other Benefits Inquiry') & ((df_main['Description'].str.contains('|'.join(['Cobra Subsidy']))) | (df_main['Subject'].str.contains('|'.join(['Cobra Subsidy'])))),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Other Benefits Inquiry') & (df_main['Type'] == 'Location'),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Other Benefits Inquiry') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Other Benefits Inquiry'),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Benefits Election Inquiry') & (df_main['Original_Subcategory__c'] == 'Benefits Election Processing') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Benefits Election Inquiry') & (df_main['Original_Subcategory__c'] == 'Benefits Election Processing'),
    (df_main['Original_Topic__c'] == 'Benefits') & ((df_main['Original_Category__c'] == 'Submit Form - Evidence of Insurability Disability/Health Statement') | (df_main['Original_Category__c'] == 'ACA Marketplace Notice')),
    (df_main['Original_Topic__c'] == 'Open Enrollment') & (df_main['Original_Category__c'] == 'Enrollment Form'),
    (df_main['Original_Subcategory__c'] == 'Employee Relations Inquiry - COE'),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'].isin(['Premium Reconciliation','Submit Form - Late Enrollment Request','UCRP/Safe Harbor Retirement Plans'])),
    (df_main['Original_Topic__c'] == 'Benefits') & (df_main['Original_Category__c'] == 'Submit Form - Newly Eligible Enrollment'),
    (df_main['Original_Topic__c'].isin(['Open Enrollment','Benefits','Compensation','Employee Relations','HR Compliance','HR Policy','Human Resources','Labor Relations','RASC','Talent Management','TAM','UCPath Online','Vendor Relations Management','Leave Balances'])) & (df_main['Submitter_ID__c'] != ''),
    (df_main['Description'].str.contains('|'.join(['expedited benefits','prescription expedited','expedited enrollment','expedited medical']))) | (df_main['Subject'].str.contains('|'.join(['expedited benefits','prescription expedited','expedited enrollment','expedited medical']))),
    (df_main['Original_Topic__c'] == 'Open Enrollment'),
    (df_main['Original_Topic__c'].isin(['Open Enrollment','Benefits','Compensation','Employee Relations','HR Compliance','HR Policy','Human Resources','Labor Relations','Leave Balances','RASC','Talent Management','TAM','UCPath Online','Vendor Relations Management'])),
    (df_main['Original_Topic__c'] == 'General Ledger') & (df_main['Original_Category__c'].isin(['Cost Recovery','Financial Control Journal'])),
    (df_main['Original_Topic__c'] == 'General Ledger'),
    (df_main['Original_Topic__c'] == 'Leaves of Absence') & (df_main['Original_Category__c'] == 'Disability / Life Insurance'),
    (df_main['Original_Topic__c'].isin(['Leaves of Absence','Mass Transactions'])),
    (df_main['Original_Topic__c'] == 'Payroll') & (df_main['Original_Category__c'].isin(['Submit Form - Payroll Administration','Submit Form - Recurring Additional Pay Correction','Payroll Administration Inquiry', 'Tax Withholding Inquiry', 'Submit Forms - Tax Forms' ])),
    (df_main['Type'] == 'Location') & (df_main['Original_Topic__c'] == 'Payroll') & (df_main['Original_Category__c'].isin(['Direct Deposit','Overpayment Recovery','Paycheck','Other Payroll Inquiry'])),
    (df_main['Original_Topic__c'] == 'UCPC Administration') & (df_main['Original_Category__c'] == 'Benefits/Vendor'),
    (df_main['Original_Topic__c'] == 'UCPC Administration') & (df_main['Original_Category__c'] == 'Benefit Eligibility Changes') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Original_Topic__c'] == 'UCPC Administration') & (df_main['Original_Category__c'] == 'Benefit Eligibility Changes'),
    (df_main['Original_Topic__c'] == 'UCPC Administration'),
    (df_main['Original_Topic__c'] == 'Workforce Administration') & (df_main['Original_Category__c'] == 'Unemployment Insurance'),
    (df_main['Original_Topic__c'] == 'Workforce Administration') & (df_main['Original_Category__c'] == 'Work Authorization / E-Verify'),
    (df_main['Original_Topic__c'] == 'Workforce Administration') & (df_main['Original_Category__c'].isin(['Submit Form - Workforce Administration','Termination/Retirement'])),
    (df_main['Original_Topic__c'] == 'Workers\' Compensation') & (df_main['Original_Category__c'].isin(['Extended Sick Leave (ESL)','Health Contingency Fund'])),
    (df_main['Original_Topic__c'] == 'Reporting') & (df_main['Original_Category__c'].isin(['Cognos - New','Cognos - Broken (Does Not Run)','Cognos - Other','Location Query'])),
    (df_main['Original_Topic__c'] == 'Reporting') & (df_main['Original_Category__c'] == 'Request a Report'),
    (df_main['Submitter_ID__c'] != ''),
    (df_main['Original_Topic__c'] == 'Reporting'),
    (df_main['Original_Topic__c'] == 'Payroll'),
    (df_main['Original_Topic__c'] == 'Records Request'),
    (df_main['Original_Topic__c'].isin(['Workforce Administration','Workers\' Compensation'])),
    (df_main['CreatedDate'] >= '1/1/2016'),



]

values = ['ES - BCC Queue',
            'ES - Settlement Agreement',
            'ES - UC Statements',
            'PROD - Payroll Data Maintenance',
            'PROD - BN Vendor Quality',
            'PROD - Former Employee',
            'ES - Former Employee',
            'ES - Verification Of Employment',
            'PROD - Quality Care Unit',
            'ES - Escalated to Sr. Agents',
            'ES - UCPath Online',
            'ES - Data Breach',
            'PROD - Quality Care Unit',
            'PROD - Escalated Support',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - WFA Escalation',
            'ITSS - GL/RPTA Escalation',
            'PROD - BN Enrollment',
            'PROD - Quality Care Unit',
            'PROD - BN Return From Leave',
            'ITSS - GL/RPTA Escalation',
            'PROD - WFA Escalation',
            'PROD - COE Escalation',
            'ES - Generalist',
            'HR - Training',
            'FIN - AR Benefits Billing',
            'PROD - BN Event Maintenance',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - Benefits',
            'ES - Trend 1',
            'ITSS - General Ledger',
            'PROD - Benefits',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Expedited Benefit Cases',
            'ES - Trend 1',
            'ES - Generalist',
            'FIN - Associates',
            'ITSS - General Ledger',
            'PROD - Benefits Absence Management',
            'PROD - Workforce Administration',
            'PROD - Payroll',
            'PROD - Payroll',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - Payroll',
            'PROD - Unemployment Insurance',
            'PROD - Audit and Compliance',
            'PROD - Workforce Administration',
            'PROD - Benefits Absence Management',
            'System Wide Reporting',
            'ITSS - Reports and Analytics',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist']

df_main['Current Routing'] = np.select(condidtions, values)
df_main['Current Routing Department'] = df_main['Current Routing'].str.split('-').str[0]

In [103]:
df_main.groupby('Current Routing').agg({'Case Number' : ['count']})

,Case Number
,count
Current Routing,
0,56
ES - BCC Queue,579
ES - Data Breach,1
ES - Escalated to Sr. Agents,16168
ES - Expedited Benefit Cases,197
ES - Former Employee,1319
ES - Generalist,61110
ES - Settlement Agreement,147


In [104]:
condidtions = [
    (df_main['Description'].str.contains('|'.join(['Death','BCC','died','Deceased','Passed Away']))) | (df_main['Subject'].str.contains('|'.join(['Death','BCC','died','Deceased','Passed Away']))),
    (df_main['Description'].str.contains('|'.join(['Settlement','Severance','arbitration','Separation Agreement']))) | (df_main['Subject'].str.contains('|'.join(['Settlement','Severance','arbitration','Separation Agreement']))) | (df_main['Category__c'] == 'Settlement Agreement'),
    (df_main['SuppliedEmail'].str.contains('|'.join(['@LFG.com','@lfg.com']))) | (df_main['SuppliedEmail'].str.contains('|'.join(['@prudential.com','@securemail-lfg.com']))) | (df_main['Origin'] == 'Email'),
    (df_main['Topic__c'] == 'Payroll') & (df_main['Category__c'] == 'Expedited Pay Request'),
    (df_main['Type'] == 'Vendor') & (df_main['Category__c'] == 'Supplemental AFLAC Plans'),
    (df_main['Type'] == 'COE') & (df_main['Category__c'] == 'Supplemental AFLAC Plans'),
    (df_main['Description'].str.contains('|'.join(['Former Employee','FEP','Stale Request','Former EE']))) | (df_main['Subject'].str.contains('|'.join(['Former Employee','FEP','Stale Request','Former EE']))),
    (df_main['Description'].str.contains('|'.join(['Cal HR','CalHR','Loan Forgiveness','WVOE','Student Verification','Perkins','employment verification','VOE','work number','PSLF','verification request', 'Income verification','TWN' ]))) | (df_main['Subject'].str.contains('|'.join(['Cal HR','CalHR','Loan Forgiveness','WVOE','Student Verification','Perkins','employment verification','VOE','work number','PSLF','verification request', 'Income verification','TWN']))),
    (df_main['Topic__c'] == 'Payroll') & (df_main['Category__c'] == 'Quality Care Unit Payroll'),
    ((df_main['Topic__c'] == 'Payroll') & (df_main['Category__c'] == 'Overpayment Recovery')) | ((df_main['Description'].str.contains('|'.join(['Overpayment','repayment agreement','repayment plan']))) | (df_main['Subject'].str.contains('|'.join(['Overpayment','repayment agreement','repayment plan'])))),
    ((df_main['Topic__c'] == 'UCPath Online') & (df_main['Category__c'].isin(['UCPath Online Access','UCPath Online Inquiry']))) | ((df_main['Description'].str.contains('|'.join(['Portal','UCPath online']))) | (df_main['Subject'].str.contains('|'.join(['Portal','UCPath online'])))),
    (df_main['Description'].str.contains('|'.join(['Accellion','Data Breach','Breach','Cyber Attack','Accellion\'s File Transfer Appliance']))) | (df_main['Subject'].str.contains('|'.join(['Accellion','Data Breach','Breach','Cyber Attack','Accellion\'s File Transfer Appliance']))),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_PYBN']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Topic__c'] == 'Payroll') & (df_main['Category__c'] == 'Dashboard/Interface Inquiry'),
    (df_main['Topic__c'] == 'Payroll') & (df_main['Category__c'] == 'Tax Withholding Inquiry') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic__c'] == 'Payroll') & (df_main['Category__c'] == 'Tax Withholding Inquiry'),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_WFA']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_GL']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Benefits Election Inquiry') & ((df_main['Description'].str.contains('|'.join(['Change election for Health FSA and/or DepCare FSA']))) | (df_main['Subject'].str.contains('|'.join(['Request COVID-19 FSA Event'])))),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Quality Care Unit Benefits'),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Submit Form - Return From Leave Benefits Reinstatement'),
    ((df_main['Topic__c'] == 'General Ledger') | (df_main['Topic__c'] == 'Reporting')) & (df_main['Category__c'] == 'Quality Care Unit Reporting'),
    (df_main['Topic__c'] == 'Workforce Administration') & (df_main['Category__c'] == 'Quality Care Unit Workforce Administration'),
    (df_main['Category__c'].isin(['Health and Welfare Benefits Inquiry','Retirement/RASC Inquiry']) ),
    (df_main['Topic__c'] == 'UCPath Help/Training Feedback') & (df_main['Category__c'] == 'General Feedback'),
    (df_main['Topic__c'] == 'UCPath Help/Training Feedback'),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Benefits Billing') & (df_main['Subcategory__c'] == 'Benefits Billing Payment Plan'),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Other Benefits Inquiry') & ((df_main['Description'].str.contains('|'.join(['Cobra Subsidy']))) | (df_main['Subject'].str.contains('|'.join(['Cobra Subsidy'])))),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Other Benefits Inquiry') & (df_main['Type'] == 'Location'),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Other Benefits Inquiry') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Other Benefits Inquiry'),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Benefits Election Inquiry') & (df_main['Subcategory__c'] == 'Benefits Election Processing') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Benefits Election Inquiry') & (df_main['Subcategory__c'] == 'Benefits Election Processing'),
    (df_main['Topic__c'] == 'Benefits') & ((df_main['Category__c'] == 'Submit Form - Evidence of Insurability Disability/Health Statement') | (df_main['Category__c'] == 'ACA Marketplace Notice')),
    (df_main['Topic__c'] == 'Open Enrollment') & (df_main['Category__c'] == 'Enrollment Form'),
    (df_main['Subcategory__c'] == 'Employee Relations Inquiry - COE'),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'].isin(['Premium Reconciliation','Submit Form - Late Enrollment Request','UCRP/Safe Harbor Retirement Plans'])),
    (df_main['Topic__c'] == 'Benefits') & (df_main['Category__c'] == 'Submit Form - Newly Eligible Enrollment'),
    (df_main['Topic__c'].isin(['Open Enrollment','Benefits','Compensation','Employee Relations','HR Compliance','HR Policy','Human Resources','Labor Relations','RASC','Talent Management','TAM','UCPath Online','Vendor Relations Management','Leave Balances'])) & (df_main['Submitter_ID__c'] != ''),
    (df_main['Description'].str.contains('|'.join(['expedited benefits','prescription expedited','expedited enrollment','expedited medical']))) | (df_main['Subject'].str.contains('|'.join(['expedited benefits','prescription expedited','expedited enrollment','expedited medical']))),
    (df_main['Topic__c'] == 'Open Enrollment'),
    (df_main['Topic__c'].isin(['Open Enrollment','Benefits','Compensation','Employee Relations','HR Compliance','HR Policy','Human Resources','Labor Relations','Leave Balances','RASC','Talent Management','TAM','UCPath Online','Vendor Relations Management'])),
    (df_main['Topic__c'] == 'General Ledger') & (df_main['Category__c'].isin(['Cost Recovery','Financial Control Journal'])),
    (df_main['Topic__c'] == 'General Ledger'),
    (df_main['Topic__c'] == 'Leaves of Absence') & (df_main['Category__c'] == 'Disability / Life Insurance'),
    (df_main['Topic__c'].isin(['Leaves of Absence','Mass Transactions'])),
    (df_main['Topic__c'] == 'Payroll') & (df_main['Category__c'].isin(['Submit Form - Payroll Administration','Submit Form - Recurring Additional Pay Correction','Payroll Administration Inquiry', 'Tax Withholding Inquiry', 'Submit Forms - Tax Forms' ])),
    (df_main['Type'] == 'Location') & (df_main['Topic__c'] == 'Payroll') & (df_main['Category__c'].isin(['Direct Deposit','Overpayment Recovery','Paycheck','Other Payroll Inquiry'])),
    (df_main['Topic__c'] == 'UCPC Administration') & (df_main['Category__c'] == 'Benefits/Vendor'),
    (df_main['Topic__c'] == 'UCPC Administration') & (df_main['Category__c'] == 'Benefit Eligibility Changes') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic__c'] == 'UCPC Administration') & (df_main['Category__c'] == 'Benefit Eligibility Changes'),
    (df_main['Topic__c'] == 'UCPC Administration'),
    (df_main['Topic__c'] == 'Workforce Administration') & (df_main['Category__c'] == 'Unemployment Insurance'),
    (df_main['Topic__c'] == 'Workforce Administration') & (df_main['Category__c'] == 'Work Authorization / E-Verify'),
    (df_main['Topic__c'] == 'Workforce Administration') & (df_main['Category__c'].isin(['Submit Form - Workforce Administration','Termination/Retirement'])),
    (df_main['Topic__c'] == 'Workers\' Compensation') & (df_main['Category__c'].isin(['Extended Sick Leave (ESL)','Health Contingency Fund'])),
    (df_main['Topic__c'] == 'Reporting') & (df_main['Category__c'].isin(['Cognos - New','Cognos - Broken (Does Not Run)','Cognos - Other','Location Query'])),
    (df_main['Topic__c'] == 'Reporting') & (df_main['Category__c'] == 'Request a Report'),
    (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic__c'] == 'Reporting'),
    (df_main['Topic__c'] == 'Payroll'),
    (df_main['Topic__c'] == 'Records Request'),
    (df_main['Topic__c'].isin(['Workforce Administration','Workers\' Compensation'])),
    (df_main['CreatedDate'] >= '1/1/2016'),



]

values = ['ES - BCC Queue',
            'ES - Settlement Agreement',
            'ES - UC Statements',
            'PROD - Payroll Data Maintenance',
            'PROD - BN Vendor Quality',
            'PROD - Former Employee',
            'ES - Former Employee',
            'ES - Verification Of Employment',
            'PROD - Quality Care Unit',
            'ES - Escalated to Sr. Agents',
            'ES - UCPath Online',
            'ES - Data Breach',
            'PROD - Quality Care Unit',
            'PROD - Escalated Support',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - WFA Escalation',
            'ITSS - GL/RPTA Escalation',
            'PROD - BN Enrollment',
            'PROD - Quality Care Unit',
            'PROD - BN Return From Leave',
            'ITSS - GL/RPTA Escalation',
            'PROD - WFA Escalation',
            'PROD - COE Escalation',
            'ES - Generalist',
            'HR - Training',
            'FIN - AR Benefits Billing',
            'PROD - BN Event Maintenance',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - Benefits',
            'ES - Trend 1',
            'ITSS - General Ledger',
            'PROD - Benefits',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Expedited Benefit Cases',
            'ES - Trend 1',
            'ES - Generalist',
            'FIN - Associates',
            'ITSS - General Ledger',
            'PROD - Benefits Absence Management',
            'PROD - Workforce Administration',
            'PROD - Payroll',
            'PROD - Payroll',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - Payroll',
            'PROD - Unemployment Insurance',
            'PROD - Audit and Compliance',
            'PROD - Workforce Administration',
            'PROD - Benefits Absence Management',
            'System Wide Reporting',
            'ITSS - Reports and Analytics',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist']

df_main['Current Final Routing'] = np.select(condidtions, values)
df_main['Current Final Routing Department'] = df_main['Current Routing'].str.split('-').str[0]

In [105]:
df_main.groupby('Current Final Routing').agg({'Case Number' : ['count']})

,Case Number
,count
Current Final Routing,
0,56
ES - BCC Queue,579
ES - Data Breach,1
ES - Escalated to Sr. Agents,13487
ES - Expedited Benefit Cases,208
ES - Former Employee,1318
ES - Generalist,57894
ES - Settlement Agreement,164


In [106]:
condidtions = [
    (df_main['Description'].str.contains('|'.join(['Death','BCC','died','Deceased','Passed Away']))) | (df_main['Subject'].str.contains('|'.join(['Death','BCC','died','Deceased','Passed Away']))),
    (df_main['Description'].str.contains('|'.join(['Settlement','Severance','arbitration','Separation Agreement']))) | (df_main['Subject'].str.contains('|'.join(['Settlement','Severance','arbitration','Separation Agreement']))) | (df_main['Category'] == 'Settlement Agreement'),
    (df_main['SuppliedEmail'].str.contains('|'.join(['@LFG.com','@lfg.com']))) | (df_main['SuppliedEmail'].str.contains('|'.join(['@prudential.com','@securemail-lfg.com']))) | (df_main['Origin'] == 'Email'),
    (df_main['Topic'] == 'Payroll') & (df_main['Category'] == 'Expedited Pay Request'),
    (df_main['Type'] == 'Vendor') & (df_main['Category'] == 'Supplemental AFLAC Plans'),
    (df_main['Type'] == 'COE') & (df_main['Category'] == 'Supplemental AFLAC Plans'),
    (df_main['Description'].str.contains('|'.join(['Former Employee','FEP','Stale Request','Former EE']))) | (df_main['Subject'].str.contains('|'.join(['Former Employee','FEP','Stale Request','Former EE']))),
    (df_main['Description'].str.contains('|'.join(['Cal HR','CalHR','Loan Forgiveness','WVOE','Student Verification','Perkins','employment verification','VOE','work number','PSLF','verification request', 'Income verification','TWN' ]))) | (df_main['Subject'].str.contains('|'.join(['Cal HR','CalHR','Loan Forgiveness','WVOE','Student Verification','Perkins','employment verification','VOE','work number','PSLF','verification request', 'Income verification','TWN']))),
    (df_main['Topic'] == 'Payroll') & (df_main['Category'] == 'Quality Care Unit Payroll'),
    ((df_main['Topic'] == 'Payroll') & (df_main['Category'] == 'Overpayment Recovery')) | ((df_main['Description'].str.contains('|'.join(['Overpayment','repayment agreement','repayment plan']))) | (df_main['Subject'].str.contains('|'.join(['Overpayment','repayment agreement','repayment plan'])))),
    ((df_main['Topic'] == 'UCPath Online') & (df_main['Category'].isin(['UCPath Online Access','UCPath Online Inquiry']))) | ((df_main['Description'].str.contains('|'.join(['Portal','UCPath online']))) | (df_main['Subject'].str.contains('|'.join(['Portal','UCPath online'])))),
    (df_main['Description'].str.contains('|'.join(['Accellion','Data Breach','Breach','Cyber Attack','Accellion\'s File Transfer Appliance']))) | (df_main['Subject'].str.contains('|'.join(['Accellion','Data Breach','Breach','Cyber Attack','Accellion\'s File Transfer Appliance']))),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_PYBN']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Topic'] == 'Payroll') & (df_main['Category'] == 'Dashboard/Interface Inquiry'),
    (df_main['Topic'] == 'Payroll') & (df_main['Category'] == 'Tax Withholding Inquiry') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic'] == 'Payroll') & (df_main['Category'] == 'Tax Withholding Inquiry'),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_WFA']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Description'].str.contains('|'.join(['voicemail_PROD_GL']))) & (df_main['SuppliedEmail'].str.contains('|'.join(['caas.inin.com']))),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Benefits Election Inquiry') & ((df_main['Description'].str.contains('|'.join(['Change election for Health FSA and/or DepCare FSA']))) | (df_main['Subject'].str.contains('|'.join(['Request COVID-19 FSA Event'])))),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Quality Care Unit Benefits'),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Submit Form - Return From Leave Benefits Reinstatement'),
    ((df_main['Topic'] == 'General Ledger') | (df_main['Topic'] == 'Reporting')) & (df_main['Category'] == 'Quality Care Unit Reporting'),
    (df_main['Topic'] == 'Workforce Administration') & (df_main['Category'] == 'Quality Care Unit Workforce Administration'),
    (df_main['Category'].isin(['Health and Welfare Benefits Inquiry','Retirement/RASC Inquiry']) ),
    (df_main['Topic'] == 'UCPath Help/Training Feedback') & (df_main['Category'] == 'General Feedback'),
    (df_main['Topic'] == 'UCPath Help/Training Feedback'),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Benefits Billing') & (df_main['SubCategory'] == 'Benefits Billing Payment Plan'),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Other Benefits Inquiry') & ((df_main['Description'].str.contains('|'.join(['Cobra Subsidy']))) | (df_main['Subject'].str.contains('|'.join(['Cobra Subsidy'])))),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Other Benefits Inquiry') & (df_main['Type'] == 'Location'),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Other Benefits Inquiry') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Other Benefits Inquiry'),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Benefits Election Inquiry') & (df_main['SubCategory'] == 'Benefits Election Processing') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Benefits Election Inquiry') & (df_main['SubCategory'] == 'Benefits Election Processing'),
    (df_main['Topic'] == 'Benefits') & ((df_main['Category'] == 'Submit Form - Evidence of Insurability Disability/Health Statement') | (df_main['Category'] == 'ACA Marketplace Notice')),
    (df_main['Topic'] == 'Open Enrollment') & (df_main['Category'] == 'Enrollment Form'),
    (df_main['SubCategory'] == 'Employee Relations Inquiry - COE'),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'].isin(['Premium Reconciliation','Submit Form - Late Enrollment Request','UCRP/Safe Harbor Retirement Plans'])),
    (df_main['Topic'] == 'Benefits') & (df_main['Category'] == 'Submit Form - Newly Eligible Enrollment'),
    (df_main['Topic'].isin(['Open Enrollment','Benefits','Compensation','Employee Relations','HR Compliance','HR Policy','Human Resources','Labor Relations','RASC','Talent Management','TAM','UCPath Online','Vendor Relations Management','Leave Balances'])) & (df_main['Submitter_ID__c'] != ''),
    (df_main['Description'].str.contains('|'.join(['expedited benefits','prescription expedited','expedited enrollment','expedited medical']))) | (df_main['Subject'].str.contains('|'.join(['expedited benefits','prescription expedited','expedited enrollment','expedited medical']))),
    (df_main['Topic'] == 'Open Enrollment'),
    (df_main['Topic'].isin(['Open Enrollment','Benefits','Compensation','Employee Relations','HR Compliance','HR Policy','Human Resources','Labor Relations','Leave Balances','RASC','Talent Management','TAM','UCPath Online','Vendor Relations Management'])),
    (df_main['Topic'] == 'General Ledger') & (df_main['Category'].isin(['Cost Recovery','Financial Control Journal'])),
    (df_main['Topic'] == 'General Ledger'),
    (df_main['Topic'] == 'Leaves of Absence') & (df_main['Category'] == 'Disability / Life Insurance'),
    (df_main['Topic'].isin(['Leaves of Absence','Mass Transactions'])),
    (df_main['Topic'] == 'Payroll') & (df_main['Category'].isin(['Submit Form - Payroll Administration','Submit Form - Recurring Additional Pay Correction','Payroll Administration Inquiry', 'Tax Withholding Inquiry', 'Submit Forms - Tax Forms' ])),
    (df_main['Type'] == 'Location') & (df_main['Topic'] == 'Payroll') & (df_main['Category'].isin(['Direct Deposit','Overpayment Recovery','Paycheck','Other Payroll Inquiry'])),
    (df_main['Topic'] == 'UCPC Administration') & (df_main['Category'] == 'Benefits/Vendor'),
    (df_main['Topic'] == 'UCPC Administration') & (df_main['Category'] == 'Benefit Eligibility Changes') & (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic'] == 'UCPC Administration') & (df_main['Category'] == 'Benefit Eligibility Changes'),
    (df_main['Topic'] == 'UCPC Administration'),
    (df_main['Topic'] == 'Workforce Administration') & (df_main['Category'] == 'Unemployment Insurance'),
    (df_main['Topic'] == 'Workforce Administration') & (df_main['Category'] == 'Work Authorization / E-Verify'),
    (df_main['Topic'] == 'Workforce Administration') & (df_main['Category'].isin(['Submit Form - Workforce Administration','Termination/Retirement'])),
    (df_main['Topic'] == 'Workers\' Compensation') & (df_main['Category'].isin(['Extended Sick Leave (ESL)','Health Contingency Fund'])),
    (df_main['Topic'] == 'Reporting') & (df_main['Category'].isin(['Cognos - New','Cognos - Broken (Does Not Run)','Cognos - Other','Location Query'])),
    (df_main['Topic'] == 'Reporting') & (df_main['Category'] == 'Request a Report'),
    (df_main['Submitter_ID__c'] != ''),
    (df_main['Topic'] == 'Reporting'),
    (df_main['Topic'] == 'Payroll'),
    (df_main['Topic'] == 'Records Request'),
    (df_main['Topic'].isin(['Workforce Administration','Workers\' Compensation'])),
    (df_main['CreatedDate'] >= '1/1/2016'),



]

values = ['ES - BCC Queue',
            'ES - Settlement Agreement',
            'ES - UC Statements',
            'PROD - Payroll Data Maintenance',
            'PROD - BN Vendor Quality',
            'PROD - Former Employee',
            'ES - Former Employee',
            'ES - Verification Of Employment',
            'PROD - Quality Care Unit',
            'ES - Escalated to Sr. Agents',
            'ES - UCPath Online',
            'ES - Data Breach',
            'PROD - Quality Care Unit',
            'PROD - Escalated Support',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - WFA Escalation',
            'ITSS - GL/RPTA Escalation',
            'PROD - BN Enrollment',
            'PROD - Quality Care Unit',
            'PROD - BN Return From Leave',
            'ITSS - GL/RPTA Escalation',
            'PROD - WFA Escalation',
            'PROD - COE Escalation',
            'ES - Generalist',
            'HR - Training',
            'FIN - AR Benefits Billing',
            'PROD - BN Event Maintenance',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - Benefits',
            'ES - Trend 1',
            'ITSS - General Ledger',
            'PROD - Benefits',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Expedited Benefit Cases',
            'ES - Trend 1',
            'ES - Generalist',
            'FIN - Associates',
            'ITSS - General Ledger',
            'PROD - Benefits Absence Management',
            'PROD - Workforce Administration',
            'PROD - Payroll',
            'PROD - Payroll',
            'PROD - Benefits',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'PROD - Payroll',
            'PROD - Unemployment Insurance',
            'PROD - Audit and Compliance',
            'PROD - Workforce Administration',
            'PROD - Benefits Absence Management',
            'System Wide Reporting',
            'ITSS - Reports and Analytics',
            'ES - Escalated to Sr. Agents',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist',
            'ES - Generalist']

df_main['Future Routing'] = np.select(condidtions, values)
df_main['Future Routing Department'] = df_main['Future Routing'].str.split('-').str[0]

In [107]:
df_main.groupby('Type').agg({'Case Number' : ['count']})

,Case Number
,count
Type,
,1569
COE,386
Employee,83370
Location,25855
UCPath Center,19939
Vendor,10485


In [108]:
df_main.groupby('Future Routing').agg({'Case Number' : ['count']})

,Case Number
,count
Future Routing,
0,43
ES - BCC Queue,579
ES - Data Breach,1
ES - Escalated to Sr. Agents,19303
ES - Expedited Benefit Cases,229
ES - Former Employee,1319
ES - Generalist,65932
ES - Settlement Agreement,134


In [109]:
df_ee = df_main.query('Type == "Employee" & Origin == "Portal"')

In [110]:
pd.set_option('display.max.rows',500)

In [111]:
df_ee.groupby('Current Routing').agg({'Case Number' : ['count']})


,Case Number
,count
Current Routing,
ES - BCC Queue,49
ES - Escalated to Sr. Agents,2128
ES - Expedited Benefit Cases,9
ES - Former Employee,7
ES - Generalist,17871
ES - Settlement Agreement,2
ES - Trend 1,3045
ES - UCPath Online,634


In [117]:
df_ee.groupby('Current Final Routing Department').agg({'Case Number' : ['count']})

,Case Number
,count
Current Final Routing Department,
ES,26484
HR,43
PROD,1387


In [112]:
df_ee.groupby('Future Routing').agg({'Case Number' : ['count']})

,Case Number
,count
Future Routing,
ES - BCC Queue,49
ES - Escalated to Sr. Agents,2230
ES - Expedited Benefit Cases,12
ES - Former Employee,7
ES - Generalist,18241
ES - Settlement Agreement,2
ES - Trend 1,2323
ES - UCPath Online,544


In [113]:
df_ee.groupby(['Current Routing','Future Routing']).agg({'Case Number' : ['count']}).sort_values(['Future Routing', 'Current Routing'], ascending=False)

,,Case Number
,,count
Current Routing,Future Routing,
PROD - Workforce Administration,PROD - Workforce Administration,43
PROD - WFA Escalation,PROD - Workforce Administration,1
PROD - Payroll,PROD - Workforce Administration,1
PROD - Benefits Absence Management,PROD - Workforce Administration,13
HR - Training,PROD - Workforce Administration,1
ES - UCPath Online,PROD - Workforce Administration,2
ES - Generalist,PROD - Workforce Administration,17
ES - Escalated to Sr. Agents,PROD - Workforce Administration,1


In [114]:
df_ee.to_csv(r'C:\Users\aquirart\Documents\Pandas\Case Assist Data\final.csv', index=False)